In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Get data

In [ ]:
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import re

import pandas as pd
import seaborn as sns
sns.set()
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import sklearn

In [ ]:
housing = pd.read_csv("../input/california-housing-prices/housing.csv")
housing.head()

# 2. Descriptive Statics

In [ ]:
housing.info()

In [ ]:
housing.describe()

In [ ]:
housing.corr(method='pearson').sort_values(by='median_income',ascending=False)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10)) 
sns.heatmap(housing.corr(), vmin=-1, vmax=1, annot = True,cmap="YlGnBu",ax=ax)

# 3. Feature engineer

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]

a=housing['median_house_value']
housing.loc[(a<=100000) , 'house_value_cat'] = 'Low' 
housing.loc[((a>100000 ) & (a <=200000)) , 'house_value_cat'] = 'Medium'  
housing.loc[((a>200000 ) & (a <=300000)) , 'house_value_cat'] = 'High'    
housing.loc[(a>300000 ) , 'house_value_cat'] = 'Very_high' 

a=housing['median_house_value']
housing.loc[(a<=100000) , 'house_value_cat'] = 0
housing.loc[((a>100000 ) & (a <=200000)) , 'house_value_cat'] = 1
housing.loc[((a>200000 ) & (a <=300000)) , 'house_value_cat'] = 2    
housing.loc[(a>300000 ) , 'house_value_cat'] = 3

# 4. Data visualization

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
housing.plot(kind='density',subplots=True,layout=(4,4),sharex=False,figsize=(10,10))
plt.tight_layout()

In [ ]:
housing.plot(kind='box', subplots=True, layout=(4,4), sharex=False,
fontsize=8,figsize=(10,10))
plt.tight_layout()

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))
plt.show()

In [ ]:
housing_group=housing.groupby('longitude').mean()
housing_group.reset_index(inplace=True)
housing_group.head()

In [ ]:
import folium

import branca
import branca.colormap as cm

x_start = (housing_group['latitude'].max() + housing_group['latitude'].min()) / 2
y_start = (housing_group['longitude'].max() + housing_group['longitude'].min()) / 2
start_coord = (x_start, y_start)

vmin=housing_group['median_house_value'].min()
vmax=housing_group['median_house_value'].max()
#colormap = cm.LinearColormap(colors=['red','lightblue'],vmin=housing_group['median_house_value'].min(),vmax=housing_group['median_house_value'].max())
colormap=cm.LinearColormap(
    ['green', 'yellow', 'red'],
    vmin=vmin, vmax=vmax)

map = folium.Map(location=start_coord, zoom_start=5)

for lat, lon, house_value, room,population in zip(housing_group['latitude'], housing_group['longitude'], 
                                       housing_group['median_house_value'],housing_group['rooms_per_household'],
                                       housing_group['population']):
  folium.CircleMarker(
        [lat, lon],
        radius=population*0.005,
                        popup = (
                                '<strong>House_value: </strong>: ' + str(house_value) + '<br>'
                                '<strong>Room: </strong>: ' + str(room) + '<br>'
                                '<strong>Population: </strong>: ' + str(population) + '<br>'),
        fill=colormap(house_value),
        color=colormap(house_value),
        fill_opacity=0.7
    ).add_to(map)
map.add_child(colormap)
map.save('map.html')
map

Map visual: radius is population (larger is higher pop), color: house value (more red more value)

In [ ]:
fig=plt.gcf()
fig.set_size_inches(10,7)
fig=sns.violinplot(x='house_value_cat',y='median_income',data=housing)

In [ ]:
sns.FacetGrid(housing, hue="house_value_cat", size=6) \
   .map(sns.kdeplot, "median_income") \
   .add_legend()
plt.ioff() 

In [ ]:
fig=sns.jointplot(x='median_income',y='median_house_value',data=housing,kind='kde',
                  color='g')

Parallel coordinate plot: This type of visualisation is used for plotting multivariate, numerical data. Parallel Coordinates Plots are ideal for comparing many variables together and seeing the relationships between them. For example, if you had to compare an array of products with the same attributes (comparing computer or cars specs across different models).

In [ ]:
import plotly.express as px
fig = px.parallel_coordinates(housing, color="house_value_cat",
                             color_continuous_scale=px.colors.diverging.Tealrose,
                             #color_continuous_midpoint=2
                              )
fig.show()

In [ ]:
import plotly.express as px
df = px.data.wind()
fig = px.line_polar(housing, r="median_house_value", theta="ocean_proximity", color="house_value_cat", line_close=True,
                    color_discrete_sequence=px.colors.sequential.Plasma_r,
                    template="plotly_dark",)
#r : do lon tren tung category
#theta: category tren radar chart
#color: mau sac
fig.show()

In [ ]:
plt.figure(figsize=(15, 25))

for i, column in enumerate(housing.columns, 1):
  plt.subplot(8, 2, i)
  housing[housing["house_value_cat"] == 0][column].hist(bins=35, color='blue', label='Low', alpha=0.6)
  housing[housing["house_value_cat"] == 1][column].hist(bins=35, color='red', label='Medium', alpha=0.6)
  housing[housing["house_value_cat"] == 2][column].hist(bins=35, color='red', label='High', alpha=0.6)
  housing[housing["house_value_cat"] == 3][column].hist(bins=35, color='red', label='VeryHigh', alpha=0.6)
  plt.legend()
  plt.xlabel(column)
plt.tight_layout

In [ ]:
px.scatter(housing, x="median_income", y="median_house_value", 
           animation_frame="ocean_proximity",
           #animation_group="age",
           size="rooms_per_household", 
           color="house_value_cat", 
           #hover_name="sex",
           #log_x=True, 
           #size_max=55, range_x=[100,100000], range_y=[25,90]
           )

In [ ]:
a=housing.sort_values(by='housing_median_age')
px.scatter(a, x="median_income", y="median_house_value", 
           animation_frame="housing_median_age",
           #animation_group="age",
           size="rooms_per_household", 
           color="house_value_cat", 
           #hover_name="sex",
           #log_x=True, 
           #size_max=55, range_x=[100,100000], range_y=[25,90]
           )